In [5]:
import os
import cv2
from cv2 import dnn_superres
import io
import imageio
import time

In [23]:
def save_to(img_arr, src_path):
    filename = f"{src_path}"
    image_file = io.BytesIO()
    imageio.imwrite(image_file, img_arr, format='jpg')
    image_file.seek(0)

    with open(filename, 'wb') as f:
        f.write(image_file.getbuffer())

In [32]:
def downscaling(image):
    rows, cols, _ = map(int, image.shape)
    return cv2.cvtColor(cv2.resize(image, ((cols // 5), (rows // 5))), cv2.COLOR_BGR2RGB)

def prepare_data(data_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".png"):
            img = cv2.cvtColor(cv2.imread(os.path.join(data_dir, filename)), cv2.COLOR_BGR2RGB)
            lr_img = downscaling(img)
            cv2.imwrite(f"{output_dir}/{filename.split('.')[0]}_5e-1x.jpg", lr_img)

In [33]:
prepare_data("images", "downscaled")

In [38]:
def preprocessing(img_path):
    filename = f"{img_path.split('/')[-1]}"
    start_time = time.time()

    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    path = 'models/fsrcnn/FSRCNN_x3.pb'
    
    superres = dnn_superres.DnnSuperResImpl_create()
    superres.readModel(path)
    superres.setModel("fsrcnn", 3)
    
    img_rgb = superres.upsample(img_rgb)
    
    # img_rgb = cv2.normalize(img_rgb, None, alpha=0, beta=320, norm_type=cv2.NORM_MINMAX)
#     img_rgb = enhance_sharpness(img_rgb, kernels['mild'])
#     img_rgb = cv2.GaussianBlur(img_rgb, (3, 3), 0)
    
    print(f"Process {img_path} in time: {round((time.time() - start_time), 2)} \n ====")
    cv2.imwrite(f"result/{filename.split('_')[0]}_fsrcnnx3.jpg", cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB))


In [39]:
preprocessing('images/example.jpg')

Process images/example.jpg in time: 59.08 
 ====
